# **영화 추천 시스템**

### 문제제기: 
기존 영화추천 시스템 프로젝트는 TMBb를 이용하고 있으며, 캐글에서 구할 수 있고, 잘 정리되어 있다는 장점이 있으나, </br>
외국 자료로, 한국의 실정과 다를 수 있다. </br>
</br>
한국인들이 평가한 한국자료로 영화 추천 시스템 현지화.

### 자료수집:
네이버 영화(https://.movie.naver.com)에서, 다음과 같은 정보를 BeautifulSoup를 이용하여 스크레이핑하였다.

- 평점 기반 랭킹 2000위(네이버에서 제공하는 수)
- 개별 페이지
  - 영화 제목
  - 영화 장르
  - 감독
  - 출연배우
  - 등급
  - 성별, 나이별 관람추이
    - 많이 본 성별(1)
    - 많이 본 나이(1)
  - 줄거리
  - 제작진(각본가)(1)

#### 기본설정

In [ ]:
#작업 경로 변경
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/codestates AIB16/project3')

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re

#### 메인페이지-랭킹 웹스크레이핑

In [ ]:
# 페이지 로드
def get_page(page_url):
  page = requests.get(page_url)
  soup = BeautifulSoup(page.content, 'html.parser')

  return soup


In [ ]:
BASE_URL = 'https://movie.naver.com/movie'

In [ ]:
# title_line.pop(0)['href']

In [ ]:
# 영화랭킹, 평점순(모든영화)
# 페이지당 50위, 총 40페이지, 2000위까지 제공
# 평점 응답자가 300명 이상인 경우 집계하고 있음(일반적인 여론조사에서 최소 응답자수: 네이버 설명)
# 20230102 기준

# 영화 제목
# 영화 평점
# 영화제목에서 movie_code

# 빈 리스트 설정
movie_code_list = []
title_list = []
get_stars = []

def get_stars_by_page_num(page_num=1):
  for p in range(1,page_num+1):
    # 연결
    rank_url = f'{BASE_URL}/sdb/rank/rmovie.naver?sel=pnt&date=20230102&page={p}'
    soup = get_page(rank_url)
    # 정보 위치 파악
    title_line = soup.select("td.title > div.tit5 > a")
    stars_list = soup.select('td.point')
    # 제목, 코드, 별점
    for x in title_line:
      title_list.append(x.text)
    for x in title_line:
      movie_code_list.append(x['href'].split('=')[-1])
    for x in stars_list:
      get_stars.append(float(x.get_text()))

    time.sleep(2)
  return title_list, movie_code_list, get_stars


In [ ]:
title_list, movie_code_list, get_stars = get_stars_by_page_num(40)
title_list

In [ ]:
len(title_list)

In [ ]:
# 데이터프레임화
df = pd.DataFrame(
      {"title":title_list,
       "code":movie_code_list,
       "stars":get_stars}  
       )

In [ ]:
df.to_csv('movie_ranking.csv',index=False, encoding='UTF-8-sig')

In [ ]:
df = pd.read_csv('movie_ranking.csv')

#### 개별 상세페이지 웹스크레이핑

In [ ]:
# 데이터 수집 계획

# 개별 페이지: df['code'] 이용하여 접근
code = df['code']
detail_url = f'{BASE_URL}/bi/mi/basic.naver?code={code}'
soup = get_page(detail_url)
info = soup.find('dl', class_='info_spec')

# 영화 장르(1)
# 감독(1)
# 출연(1)
# 등급
genre = info.select('a:nth-of-type(1)').pop().text
director = info.select('a:nth-of-type(5)').pop().text.replace(' ','')
actor1 = info.select('a:nth-of-type(6)').pop().text.replace(' ','')
# 구글 코랩은 nth-of-type만 지원한다.
# last-of-type, nth-last-of-type 모두 오류..
# grade = info.select('a:nth-last-of-type(1)').pop().text.replace(' ','')

# 성별, 나이별 관람추이
  # 많이 본 성별(1)
   # donut_graph내에 값에 find, select로 접근이 안된다..
   # 자바스크립트로 동적으로 불러온다고 한다. 셀레니움 또는 disable JS 후 이미지 자체 추출
  # 많이 본 나이(1) - EDA하면서 new_column 추출
age10 = soup.select('div.bar_graph strong:nth-of-type(1)').pop()
age20 = soup.select('div.bar_graph strong:nth-of-type(3)').pop()
age30 = soup.select('div.bar_graph strong:nth-of-type(5)').pop()
age40 = soup.select('div.bar_graph strong:nth-of-type(7)').pop()
age50 = soup.select('div.bar_graph strong:nth-of-type(9)').pop()

age10_percent = re.sub('[^0-9]','',str(age10))
age20_percent = re.sub('[^0-9]','',str(age20))
age30_percent = re.sub('[^0-9]','',str(age30))
age40_percent = re.sub('[^0-9]','',str(age40))
age50_percent = re.sub('[^0-9]','',str(age50))
# 줄거리: skip
# 제작진(각본가)(1)
super_detail_url = f'{BASE_URL}/movie/bi/mi/detail.naver?code={code}'
staff = re.sub('[^가-힣]','',str(soup.select_one('table.staff_lst td > span')))

In [ ]:
df_added = df.copy()
df_added

In [ ]:
# 영화장르, 감독, 주연
genre_list = []
director_list = []
actor_list = []

for i in range(0,2000):
  code = df_added['code'][i]
  detail_url = f'{BASE_URL}/bi/mi/basic.naver?code={code}'
  soup = get_page(detail_url)
  info = soup.find('dl', class_='info_spec')

  try:
    info.select('a:nth-of-type(1)').pop()
  except:
    genre_list.append(None)
  else:
    genre_list.append(info.select('a:nth-of-type(1)').pop().text)

  try:
    info.select('p:nth-of-type(2) > a:nth-of-type(1)').pop()
  except:
    director_list.append(None)
  else:
    director_list.append(info.select('p:nth-of-type(2) > a:nth-of-type(1)').pop().text.replace(' ',''))

  try:
    info.select('p:nth-of-type(3) > a:nth-of-type(1)').pop()
  except:
    actor_list.append(None)
  else:
    actor_list.append(info.select('p:nth-of-type(3) > a:nth-of-type(1)').pop().text.replace(' ',''))

  time.sleep(1)


  # genre_list.append(info.select('a:nth-of-type(1)').pop().text)
  # director_list.append(info.select('a:nth-of-type(5)').pop().text.replace(' ',''))
  # actor_list.append(info.select('a:nth-of-type(6)').pop().text.replace(' ',''))




In [ ]:
# 단어 유사도 분석 시 떨어지지 않는 한 단어 자체의 중요도를 갖기 위해서는 띄어쓰기를 없애주어야 한다.
# 띄어쓰기를 기준으로 단어 분리.

In [ ]:
# test
code = 160399
detail_url = f'{BASE_URL}/bi/mi/basic.naver?code={code}'
soup2 = get_page(detail_url)
info2 = soup2.find('dl', class_='info_spec')
info2.select('a')

In [ ]:
df_info = pd.DataFrame(
    {"genre":genre_list,
     "director":director_list,
     "actor":actor_list}
     )

In [ ]:
df_info.to_csv('movie_info.csv',index=False, encoding='UTF-8-sig')

In [ ]:
#나이대별 관람비율
age10_list = []
age20_list = []
age30_list = []
age40_list = []
age50_list = []

for i in range(0,1999):
  code = df_added['code'][i]
  detail_url = f'{BASE_URL}/bi/mi/basic.naver?code={code}'
  soup = get_page(detail_url)
  info = soup.find('dl', class_='info_spec')

  try:
    age10 = soup.select('div.bar_graph strong:nth-of-type(1)').pop()
  except:
    age10_list.append('none')
  else:
    age10_list.append(re.sub('[^0-9]','',str(age10)))

  try:
    age20 = soup.select('div.bar_graph strong:nth-of-type(3)').pop()
  except:
    age20_list.append('none')
  else:
    age20_list.append(re.sub('[^0-9]','',str(age20)))

  try:
    age30 = soup.select('div.bar_graph strong:nth-of-type(5)').pop()
  except:
    age30_list.append('none')
  else:
    age30_list.append(re.sub('[^0-9]','',str(age30)))

  try:
    age40 = soup.select('div.bar_graph strong:nth-of-type(7)').pop()
  except:
    age40_list.append('none')
  else:
    age40_list.append(re.sub('[^0-9]','',str(age40)))

  try:
    age50 = soup.select('div.bar_graph strong:nth-of-type(9)').pop()
  except:
    age50_list.append('none')
  else:
    age50_list.append(re.sub('[^0-9]','',str(age50)))

  time.sleep(1)


    # age10 = soup.select('div.bar_graph strong:nth-of-type(1)').pop()
    # age10_list.append(re.sub('[^0-9]','',str(age10)))
    # age20 = soup.select('div.bar_graph strong:nth-of-type(3)').pop()
    # age20_list.append(re.sub('[^0-9]','',str(age20)))
    # age30 = soup.select('div.bar_graph strong:nth-of-type(5)').pop()
    # age30_list.append(re.sub('[^0-9]','',str(age30)))
    # age40 = soup.select('div.bar_graph strong:nth-of-type(7)').pop()
    # age40_list.append(re.sub('[^0-9]','',str(age40)))
    # age50 = soup.select('div.bar_graph strong:nth-of-type(9)').pop()
    # age50_list.append(re.sub('[^0-9]','',str(age50)))
    # IndexError: pop from empty list 해당 정보가 없는 경우(bar_graph자체가 없음)가 있어 오류가 난다. -> 예외처리


  # df_added['genre'][i] = info.select('a:nth-of-type(1)').pop().text
  # df_added['director'][i] = info.select('a:nth-of-type(5)').pop().text.replace(' ','')
  # df_added['actor'][i] = info.select('a:nth-of-type(6)').pop().text.replace(' ','')

  # age10 = soup.select('div.bar_graph strong:nth-of-type(1)').pop()
  # df_added['age10'][i] = re.sub('[^0-9]','',str(age10))
  # age20 = soup.select('div.bar_graph strong:nth-of-type(3)').pop()
  # df_added['age20'][i] = re.sub('[^0-9]','',str(age20))
  # age30 = soup.select('div.bar_graph strong:nth-of-type(5)').pop()
  # df_added['age30'][i] = re.sub('[^0-9]','',str(age30))
  # age40 = soup.select('div.bar_graph strong:nth-of-type(7)').pop()
  # df_added['age40'][i] = re.sub('[^0-9]','',str(age40))
  # age50 = soup.select('div.bar_graph strong:nth-of-type(9)').pop()
  # df_added['age50'][i] = re.sub('[^0-9]','',str(age50))

  # error message: 
  # if is_scalar(key) and isna(key) and not self.hasnans: KeyError: 'genre'
  # 데이터프레임에서 한 열의 정보를 바탕으로 다른열을 추가할 수 없나?

In [ ]:
df_age = pd.DataFrame(
    {"age10":age10_list,
     "age20":age20_list,
     "age30":age30_list,
     "age40":age40_list,
     "age50":age50_list}
     )

In [ ]:
# idxmax를 쓰기 위해.. 정수타입으로 변환
# 처음부터 어떻게 쓸 것이고, 어떤 타입이어야 하는지 잘 설계해야 한다.
df['age10'] = df['age10'].str.replace('none', '0')
df['age20'] = df['age20'].str.replace('none', '0')
df['age30'] = df['age30'].str.replace('none', '0')
df['age40'] = df['age40'].str.replace('none', '0')
df['age50'] = df['age50'].str.replace('none', '0')

df['age10'] = df['age10'].astype(int)
df['age20'] = df['age20'].astype(int)
df['age30'] = df['age30'].astype(int)
df['age40'] = df['age40'].astype(int)
df['age50'] = df['age50'].astype(int)

In [ ]:
#가장 높은 퍼센티지의 값
high_list = []
for i in list(df.index):
  high_list.append(df.iloc[i].max())

high_list

In [ ]:
df['highest_age'] = df.T.idxmax() #열 중 가장 높은 값의 index(행번호) 호출. 그래서 T로 뒤집어서 진행한 후 다시 붙이기
df['high_score'] = high_list

In [ ]:
for i in range(0, len(df['highest_age'])):
  if df['high_score'][i] == 0:
    df['highest_age'][i] = None 
#idxmax가 같은 값이면 먼저 나오는 값을 주기 때문에 모두가 0인 경우(집계없는 경우)에는 highest_age도 없애기

In [ ]:
df_age_percentages= df[['high_score', 'highest_age']] # 나이별 세부정보 제외
df_age_percentages.rename(columns={'high_score':'highest_percentages'}, inplace=True) #이해하기 쉽도록 컬럼명 변경
df_age_percentages

In [ ]:
# csv로 저장
df_age_percentages.to_csv('movie_age_percentages_cleaned.csv', index=False)

In [ ]:
# 스태프-각본
# 다큐일 경우 각본가 대신 프로듀서가 처음으로 써져있는 경우가 있는데, 
# EDA하면서 '각본'이 아닌 경우 drop..? 
staff_list = []

for i in range(0,1999):
  code = df_added['code'][i]
  super_detail_url = f'{BASE_URL}/movie/bi/mi/detail.naver?code={code}'
  soup = get_page(super_detail_url)

  try: 
    soup.select_one('div.staff > table.staff_lst td > span')
  except:
    staff_list.append('none')
  else:
    staff_list.append(re.sub('[^가-힣]','',str(soup.select_one('div.staff > table.staff_lst td > span'))))

  time.sleep(1)


In [ ]:
df_scenario = pd.DataFrame({'writer':staff_list})

In [ ]:
df_scenario.to_csv('movie_writer.csv', index=False)

### 스크레이핑 완료, 데이터 정제
age의 경우 위에서 수집하면서 특성공학 함께 진행한 상태

In [ ]:
# 각 csv파일 가져오기
# df_movie: 영화 타이틀, 영화코드, 별점 정보('movie_ranking.csv')
# df_info: 영화 장르, 감독, 배우('movie_info.csv')
# df_writer: 영화 각본('movie_writer.csv')
# df_age: 해당 영화를 즐겨본 나이('movie_age_percentages_cleaned.csv')(range작게 넣어서 누락되었던 마지막 줄 추가한 파일)

In [ ]:
df_movie = pd.read_csv('movie_ranking.csv')
df_info = pd.read_csv('movie_info.csv')
df_writer = pd.read_csv('movie_writer.csv')
df_age = pd.read_csv('movie_age_percentages_cleaned.csv')

In [ ]:
df_info.info() 
# 비는 건 19금으로 로그인하고 인증 후 접근 가능..

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   genre     1976 non-null   object
 1   director  1976 non-null   object
 2   actor     1976 non-null   object
dtypes: object(3)
memory usage: 47.0+ KB


In [ ]:
df_writer.info() #소규모 독립영화, 다큐멘터리영화의 경우 없을 수 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   writer  1922 non-null   object
dtypes: object(1)
memory usage: 15.8+ KB


In [ ]:
m_df = pd.concat([df_movie, df_info, df_writer, df_age], axis=1)
m_df

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,탑건: 매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,age30
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,NaN,0,NaN
2,인생은 뷰티풀: 비타돌체,213364,9.66,공연실황,김선형,김호중,NaN,58,age50
3,할머니의 먼 집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,age20
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,age20
...,...,...,...,...,...,...,...,...,...
1995,링컨 차를 타는 변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,NaN
1996,해피 데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,age20
1997,하트 오브 더 씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,age20
1998,기억의 밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,age20


In [ ]:
m_df.to_csv('movie_nv.csv',index=False, encoding='UTF-8-sig')

In [ ]:
movie_df = pd.read_csv('movie_nv.csv')
movie_df

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,탑건: 매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,age30
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,NaN,0,NaN
2,인생은 뷰티풀: 비타돌체,213364,9.66,공연실황,김선형,김호중,NaN,58,age50
3,할머니의 먼 집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,age20
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,age20
...,...,...,...,...,...,...,...,...,...
1995,링컨 차를 타는 변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,NaN
1996,해피 데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,age20
1997,하트 오브 더 씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,age20
1998,기억의 밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,age20


In [ ]:
movie_df.describe(include='O')

,title,genre,director,actor,writer,highest_age
count,2000,1976,1976,1976,1922,1048
unique,1982,22,1224,1083,1393,5
top,천국의 아이들,드라마,스티븐스필버그,톰행크스,미야자키하야오각본,age20
freq,2,644,18,20,11,751


In [ ]:
movie_df[movie_df['code'].duplicated()]

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age


In [ ]:
movie_df[movie_df['title']=='천국의 아이들']

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
152,천국의 아이들,29571,9.28,드라마,마지드마지디,레자나지,마지드마지디각본,60,age20
1116,천국의 아이들,88474,8.70,드라마,박흥식,유다인,조정호프로듀서,0,NaN


In [ ]:
movie_df['title'] = movie_df['title'].str.replace(' ','')

In [ ]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                2000 non-null   object 
 1   code                 2000 non-null   int64  
 2   stars                2000 non-null   float64
 3   genre                1976 non-null   object 
 4   director             1976 non-null   object 
 5   actor                1976 non-null   object 
 6   writer               1922 non-null   object 
 7   highest_percentages  2000 non-null   int64  
 8   highest_age          1048 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 140.8+ KB


In [ ]:
movie_df_dropped = movie_df.dropna(subset=['genre'])
# 성인인증 필요한 데이터 삭제

In [ ]:
movie_df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                1976 non-null   object 
 1   code                 1976 non-null   int64  
 2   stars                1976 non-null   float64
 3   genre                1976 non-null   object 
 4   director             1976 non-null   object 
 5   actor                1976 non-null   object 
 6   writer               1922 non-null   object 
 7   highest_percentages  1976 non-null   int64  
 8   highest_age          1048 non-null   object 
dtypes: float64(1), int64(2), object(6)
memory usage: 154.4+ KB


In [ ]:
movie_df_none = movie_df_dropped.copy()
movie_df_none['writer'] = movie_df_dropped['writer'].fillna('None')
movie_df_none

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,age30
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,NaN
2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,age50
3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,age20
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,age20
...,...,...,...,...,...,...,...,...,...
1995,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,NaN
1996,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,age20
1997,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,age20
1998,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,age20


In [ ]:
movie_df_not_s = movie_df_none.copy()
movie_df_not_s['highest_age'] = movie_df_none['highest_age'].fillna(0)
movie_df_not_s

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,age30
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,0
2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,age50
3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,age20
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,age20
...,...,...,...,...,...,...,...,...,...
1995,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,0
1996,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,age20
1997,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,age20
1998,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,age20


In [ ]:
movie_df_not_s = movie_df_not_s.replace({'highest_age':'age10'}, 10)
movie_df_not_s = movie_df_not_s.replace({'highest_age':'age20'}, 20)
movie_df_not_s = movie_df_not_s.replace({'highest_age':'age30'}, 30)
movie_df_not_s = movie_df_not_s.replace({'highest_age':'age40'}, 40)
movie_df_not_s = movie_df_not_s.replace({'highest_age':'age50'}, 50)
movie_df_not_s

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,30
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,0
2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,50
3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,20
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,20
...,...,...,...,...,...,...,...,...,...
1995,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,0
1996,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,20
1997,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,20
1998,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,20


In [ ]:
movie_df_not_s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1976 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   title                1976 non-null   object 
 1   code                 1976 non-null   int64  
 2   stars                1976 non-null   float64
 3   genre                1976 non-null   object 
 4   director             1976 non-null   object 
 5   actor                1976 non-null   object 
 6   writer               1976 non-null   object 
 7   highest_percentages  1976 non-null   int64  
 8   highest_age          1976 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 154.4+ KB


In [ ]:
df=movie_df_not_s

In [ ]:
df.to_csv('movie_df_cleaned.csv', index=False, encoding='UTF-8-sig')

### 파일 내 데이터 인덱스 정리, Bag of Words 만들어주기

In [ ]:
df = pd.read_csv('movie_df_cleaned.csv')
df

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,30
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,0
2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,50
3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,20
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,20
...,...,...,...,...,...,...,...,...,...
1971,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,0
1972,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,20
1973,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,20
1974,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,20


In [ ]:
df.reset_index()
# 성인인증 삭제하면서 인덱스 비는 번호 생김

,index,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age
0,0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,30
1,1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,0
2,2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,50
3,3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,20
4,4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,20
...,...,...,...,...,...,...,...,...,...,...
1971,1971,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,0
1972,1972,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,20
1973,1973,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,20
1974,1974,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,20


In [ ]:
def create_soup(x):
  return x['genre']+' '+x['director']+' '+x['actor']+' '+x['writer']
df['soup'] = df.apply(create_soup, axis=1)
df

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age,soup
0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,30,액션 조셉코신스키 톰크루즈 피터크레이그각본
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,0,애니메이션 서지오파블로스 제이슨슈왈츠먼 None
2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,50,공연실황 김선형 김호중 None
3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,20,다큐멘터리 이소현 박삼순 안보영프로듀서
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,20,다큐멘터리 박혜령 임지호 엄정화프로듀서
...,...,...,...,...,...,...,...,...,...,...
1971,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,0,범죄 브래드퍼맨 매튜맥커너히 존로마노각본
1972,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,20,미스터리 크리스토퍼랜던 제시카로테 스콧로브델각본
1973,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,20,액션 론하워드 크리스헴스워스 찰스리빗각본
1974,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,20,미스터리 장항준 강하늘 장항준각본


In [ ]:
df.to_csv('movie_df_soup.csv', index=False, encoding='UTF-8-sig')

### 단어 벡터라이즈, 단어유사도 구하기

In [ ]:
# #한국어 토크나이저
# !pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 39.7 MB/s eta 0:00:00


In [ ]:
# from konlpy.tag import Okt
# okt=Okt()

# countervectorizer가 영어에 최적화되어 있어 한국어 분석 패키지 써야하나 했는데, 조사 등의 제거가 불필요한 단어 단위의 데이터이고, countvectorizer가 잘 동작했다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
count_matrix = count.fit_transform(df['soup'])
count_matrix

<1976x3690 sparse matrix of type '<class 'numpy.int64'>'
	with 8022 stored elements in Compressed Sparse Row format>

In [ ]:
# 단어 유사도 비교
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.  , 0.  , 0.  , ..., 0.25, 0.  , 0.  ],
       [0.  , 1.  , 0.25, ..., 0.  , 0.  , 0.  ],
       [0.  , 0.25, 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.25, 0.  , 0.  , ..., 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 0.  , 1.  ]])

In [ ]:
cosine_sim.shape

(1976, 1976)

In [ ]:
indices = pd.Series(df.index, index=df['title'])
indices

title
탑건:매버릭              0
클라우스                1
인생은뷰티풀:비타돌체         2
할머니의먼집              3
밥정                  4
                 ... 
링컨차를타는변호사        1971
해피데스데이           1972
하트오브더씨           1973
기억의밤             1974
터미네이터:미래전쟁의시작    1975
Length: 1976, dtype: int64

In [ ]:
indices['소원'] # 영화제목으로 인덱스 호출

102

In [ ]:
df.iloc[[102]] # 인덱스로 정보 호출

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age,soup
102,소원,103535,9.33,드라마,이준익,설경구,김지혜각본,0,0,드라마 이준익 설경구 김지혜각본


### 영화추천 함수

In [ ]:
# 영화의 제목을 입력받으면 코사인 유사도를 통해 가장 유사도가 높은 상위 3개의 영화 목록 반환
def get_recommendations(title, cosine_sim=cosine_sim):
  # 영화 제목을 통해서 전체 데이터 기준 그 영화의 index값 얻기
  idx = indices[title]
  # 코사인 유사도 매트릭스(cosine_sim)에서 idx(영화[102])에 해당하는 데이터를 (idx, 유사도)형태(enumerate)로 얻기
  sim_scores = list(enumerate(cosine_sim[idx]))
  # 유사도 기준 내림차순하여 가장 유사도가 큰 영화
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) #람다, x의 [1]번째 반환
  # 첫번째[0]인 1값은 자기자신으로 제외
  sim_scores = sim_scores[1:4]
  # idx,유사도값만 보이므로 idx로 영화 제목 가져오기
  movie_indices = [i[0] for i in sim_scores]
  
  return df['title'].iloc[movie_indices]


In [ ]:
list(enumerate(cosine_sim[102])) # 코사인 유사도 매트릭스(cosine_sim)에서 idx(영화[102])에 해당하는 데이터를 (idx, 유사도)형태(enumerate)로 얻기

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.25),
 (6, 0.25),
 (7, 0.0),
 (8, 0.25),
 (9, 0.25),
 (10, 0.0),
 (11, 0.0),
 (12, 0.25),
 (13, 0.0),
 (14, 0.25),
 (15, 0.0),
 (16, 0.0),
 (17, 0.25),
 (18, 0.0),
 (19, 0.25),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.25),
 (29, 0.25),
 (30, 0.0),
 (31, 0.0),
 (32, 0.25),
 (33, 0.0),
 (34, 0.20412414523193154),
 (35, 0.0),
 (36, 0.0),
 (37, 0.25),
 (38, 0.25),
 (39, 0.0),
 (40, 0.0),
 (41, 0.0),
 (42, 0.0),
 (43, 0.25),
 (44, 0.0),
 (45, 0.0),
 (46, 0.25),
 (47, 0.22360679774997896),
 (48, 0.0),
 (49, 0.25),
 (50, 0.0),
 (51, 0.0),
 (52, 0.25),
 (53, 0.0),
 (54, 0.25),
 (55, 0.0),
 (56, 0.5),
 (57, 0.0),
 (58, 0.0),
 (59, 0.0),
 (60, 0.0),
 (61, 0.0),
 (62, 0.0),
 (63, 0.25),
 (64, 0.25),
 (65, 0.0),
 (66, 0.25),
 (67, 0.0),
 (68, 0.0),
 (69, 0.0),
 (70, 0.0),
 (71, 0.0),
 (72, 0.75),
 (73, 0.0),
 (74, 0.0),
 (75, 0.0),
 (76, 0.0),
 (77, 0.25),
 (78, 0.0),
 (79, 0

In [ ]:
sorted(sim_sc, key=lambda x: x[1], reverse=True)

[(102, 1.0),
 (72, 0.75),
 (56, 0.5),
 (308, 0.5),
 (500, 0.5),
 (583, 0.5),
 (1792, 0.5),
 (1936, 0.5),
 (1757, 0.2886751345948129),
 (5, 0.25),
 (6, 0.25),
 (8, 0.25),
 (9, 0.25),
 (12, 0.25),
 (14, 0.25),
 (17, 0.25),
 (19, 0.25),
 (28, 0.25),
 (29, 0.25),
 (32, 0.25),
 (37, 0.25),
 (38, 0.25),
 (43, 0.25),
 (46, 0.25),
 (49, 0.25),
 (52, 0.25),
 (54, 0.25),
 (63, 0.25),
 (64, 0.25),
 (66, 0.25),
 (77, 0.25),
 (81, 0.25),
 (89, 0.25),
 (90, 0.25),
 (91, 0.25),
 (94, 0.25),
 (95, 0.25),
 (98, 0.25),
 (108, 0.25),
 (110, 0.25),
 (112, 0.25),
 (114, 0.25),
 (116, 0.25),
 (123, 0.25),
 (135, 0.25),
 (136, 0.25),
 (137, 0.25),
 (140, 0.25),
 (144, 0.25),
 (147, 0.25),
 (149, 0.25),
 (152, 0.25),
 (153, 0.25),
 (158, 0.25),
 (159, 0.25),
 (160, 0.25),
 (164, 0.25),
 (170, 0.25),
 (171, 0.25),
 (174, 0.25),
 (175, 0.25),
 (186, 0.25),
 (187, 0.25),
 (189, 0.25),
 (193, 0.25),
 (194, 0.25),
 (196, 0.25),
 (197, 0.25),
 (199, 0.25),
 (200, 0.25),
 (204, 0.25),
 (205, 0.25),
 (206, 0.25),
 (2

In [ ]:
get_recommendations('소원')

72     자산어보
56       동주
308    오아시스
Name: title, dtype: object

In [ ]:
df.iloc[[102, 72, 56, 308]]

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age,soup
102,소원,103535,9.33,드라마,이준익,설경구,김지혜각본,0,0,드라마 이준익 설경구 김지혜각본
72,자산어보,189075,9.35,드라마,이준익,설경구,김세겸각본,30,30,드라마 이준익 설경구 김세겸각본
56,동주,134899,9.37,드라마,이준익,강하늘,신연식각본,55,20,드라마 이준익 강하늘 신연식각본
308,오아시스,34093,9.18,드라마,이창동,설경구,이창동각본,67,20,드라마 이창동 설경구 이창동각본


### 피클

In [ ]:
import pickle

In [ ]:
# pickle.dump(df, open('movies.pickle', 'wb')) #vscode에서 streamlit 쓸 때 이 형식에서 계속 오류가 났다. 저장이 안됐다기엔 다시 불러오기하면 잘 불러와진다.

In [ ]:
# pickle.dump(cosine_sim, open('cosine_sim.pickle', 'wb'))

In [ ]:
with open('movi.pkl', 'wb') as mv:
  pickle.dump(df, mv)

In [ ]:
with open('sim.pkl', 'wb') as sim:
  pickle.dump(cosine_sim, sim)

### 피클 확인

In [ ]:
with open('movies.pickle', 'rb') as p:
  list_movie = pickle.load(p) 

In [ ]:
list_movie

,title,code,stars,genre,director,actor,writer,highest_percentages,highest_age,soup
0,탑건:매버릭,81888,9.76,액션,조셉코신스키,톰크루즈,피터크레이그각본,37,30,액션 조셉코신스키 톰크루즈 피터크레이그각본
1,클라우스,191613,9.67,애니메이션,서지오파블로스,제이슨슈왈츠먼,None,0,0,애니메이션 서지오파블로스 제이슨슈왈츠먼 None
2,인생은뷰티풀:비타돌체,213364,9.66,공연실황,김선형,김호중,None,58,50,공연실황 김선형 김호중 None
3,할머니의먼집,144584,9.62,다큐멘터리,이소현,박삼순,안보영프로듀서,68,20,다큐멘터리 이소현 박삼순 안보영프로듀서
4,밥정,186114,9.61,다큐멘터리,박혜령,임지호,엄정화프로듀서,35,20,다큐멘터리 박혜령 임지호 엄정화프로듀서
...,...,...,...,...,...,...,...,...,...,...
1971,링컨차를타는변호사,80626,8.19,범죄,브래드퍼맨,매튜맥커너히,존로마노각본,0,0,범죄 브래드퍼맨 매튜맥커너히 존로마노각본
1972,해피데스데이,164932,8.19,미스터리,크리스토퍼랜던,제시카로테,스콧로브델각본,71,20,미스터리 크리스토퍼랜던 제시카로테 스콧로브델각본
1973,하트오브더씨,114225,8.19,액션,론하워드,크리스헴스워스,찰스리빗각본,50,20,액션 론하워드 크리스헴스워스 찰스리빗각본
1974,기억의밤,160399,8.19,미스터리,장항준,강하늘,장항준각본,62,20,미스터리 장항준 강하늘 장항준각본


### 구현 테스트

In [ ]:
with open('movi.pkl', 'rb') as mv:
    movies = pickle.load(mv)
with open('sim.pkl', 'rb') as s:
    cosine_sim = pickle.load(s)

In [ ]:
def get_recommendations(title):
    # 영화 제목을 통해 전체 데이터 기준 그 영화의 index 얻기
    idx = movies[movies['title'] == title].index[0]
    # 코사인 유사도 매트릭스에서 위 인덱스 기준의 데이터 찾기
    sim_scores = list(enumerate(cosine_sim[idx]))
    # 유사도 내림차순
    sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # 자신을 제외한 상위 유사도 4개 슬라이싱
    sim_scores = sim_scores[1:5]
    # 슬라이싱한 4개 유사도의 인덱스 추출
    movie_indices = [i[0] for i in sim_scores]
    # 인덱스 정보로 영화정보 추출
    titles = []
    stars = []
    directors = []
    main_actors = []
    genres = []
    like_ages = []

    for i in movie_indices:
        titles.append(movies['title'].iloc[i])
        stars.append(movies['stars'].iloc[i])
        genres.append(movies['genre'].iloc[i])
        directors.append(movies['director'].iloc[i])
        main_actors.append(movies['actor'].iloc[i])
        like_ages.append(movies['highest_age'].iloc[i])
        
    return titles, stars, genres, directors, main_actors, like_ages


In [ ]:
get_recommendations('소원') # 상위4개가 아닌 전체리스트에서 랭킹 상위 4개 소환되어, 한 줄 씩 확인해보았다.

(['클라우스', '인생은뷰티풀:비타돌체', '할머니의먼집', '밥정'],
 [9.67, 9.66, 9.62, 9.61],
 ['애니메이션', '공연실황', '다큐멘터리', '다큐멘터리'],
 ['서지오파블로스', '김선형', '이소현', '박혜령'],
 ['제이슨슈왈츠먼', '김호중', '박삼순', '임지호'],
 [0, 50, 20, 20])

In [ ]:
# 영화 제목을 통해 전체 데이터 기준 그 영화의 index 얻기
idx = movies[movies['title'] == '소원'].index[0]
idx
# # 코사인 유사도 매트릭스에서 위 인덱스 기준의 데이터 찾기
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores
# # 유사도 내림차순
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # sorted는 그렇게 보이기는 하지만, 그 데이터 자체를 변환시키는 것이 아니라서 다시 저장해주어야한다.
sim_scores
# # 자신을 제외한 상위 유사도 4개 슬라이싱
sim_scores = sim_scores[1:5]
sim_scores
# # 슬라이싱한 4개 유사도의 인덱스 추출
movie_indices = [i[0] for i in sim_scores]
movie_indices

[72, 56, 308, 500]